In [3]:
import pandas as pd
import mysql.connector

# Establishing a MySQL connection
connection = mysql.connector.connect(
    host='localhost',
    user="root",
    password="KpakpoParker700!",
    database="chaku_foods"
)
cursor = connection.cursor()

# Load the Excel file
excel_file = r"C:\Users\CHAKU FOODS\Documents\Farms Table.xlsx"
excel_data = pd.ExcelFile(excel_file)

# Modified helper function to get farmer_id without normalizing names
def get_farmer_id(first_name, last_name):
    if pd.isna(first_name) or pd.isna(last_name):
        return None
    query = "SELECT farmer_id FROM farmer WHERE first_name = %s AND last_name = %s"
    cursor.execute(query, (first_name, last_name))
    result = cursor.fetchone()
    if not result:
        print(f"Farmer not found: {first_name} {last_name}")
    return result[0] if result else None

# Modified helper function to insert data into MySQL
def insert_into_mysql(table_name, data):
    for index, row in data.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(row.index)
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(sql, tuple(None if pd.isna(value) else value for value in row))
    connection.commit()

# Columns expected in the farm table (excluding geo_coordinates and geo_boundaries)
farm_table_columns = [
    "farmer_id",
    "farm_number",
    "farm_location",
    "road_accessibility"
]

# Process the Excel sheets to find data relevant to the farm table
farm_data = []
for sheet_name in excel_data.sheet_names:
    print(f"Processing sheet: {sheet_name}")
    df = excel_data.parse(sheet_name)

    # Normalize column names
    df.columns = [col.strip() for col in df.columns]

    # Check if the sheet has the necessary columns
    relevant_columns = [col for col in farm_table_columns if col != "farmer_id" and col in df.columns]

    if "last_name" in df.columns and "first_name" in df.columns:
        # Resolve farmer_id using names without normalization
        df["farmer_id"] = df.apply(
            lambda row: get_farmer_id(row["first_name"], row["last_name"]) \
            if pd.notnull(row["first_name"]) and pd.notnull(row["last_name"]) else None,
            axis=1
        )

        # Filter rows with valid farmer_id
        valid_rows = df[df["farmer_id"].notnull()]

        # Prepare the data for insertion
        for _, row in valid_rows.iterrows():
            farm_entry = {
                "farmer_id": row["farmer_id"],
            }
            for col in relevant_columns:
                farm_entry[col] = row.get(col, None)
            farm_data.append(farm_entry)
    else:
        print(f"Sheet '{sheet_name}' is missing 'First Name' or 'Last Name' columns.")

# Convert farm data to DataFrame and insert into MySQL
if farm_data:
    farm_df = pd.DataFrame(farm_data)
    insert_into_mysql('farm', farm_df)
else:
    print("No valid data found for the farm table.")

# Close the database connection
cursor.close()
connection.close()

print("Farm table data import completed successfully.")


Processing sheet: Farm Table
Farm table data import completed successfully.
